In [38]:
import json
import pandas as pd
from pandas import json_normalize

# Load the JSON data
with open('C:/Users/aober/Documents/Data Science Studium/3Semester/SoftwareProject3/PM3_1/data/masterjson.json', 'r') as file:
    data = json.load(file)


In [39]:
for item in data['results']:
    # Merge the attributes from 'attrs' into the main item
    item.update(item['attrs'])
    # Delete the now unnecessary keys
    del item['attrs']
    del item['id']
    del item['weight']
    del item['geom_quadindex']
    del item['geom_st_box2d']
    del item['origin']
    del item['rank']
    del item['x']
    del item['y']
    del item['zoomlevel']
    del item['num']
    del item['detail']

In [40]:
nodes_data = data['results']

# Convert to DataFrame
df = pd.DataFrame(nodes_data)

print(df.columns)


Index(['label', 'lat', 'lon', 'objectclass', 'Kategorie', 'schoolshort',
       'name', 'quarter', 'city', 'address', 'zipcode'],
      dtype='object')


In [52]:
data_dict = df.to_dict(orient='records')

# Specify the path where you want to save the JSON file
output_path = 'C:/Users/aober/Documents/Data Science Studium/3Semester/SoftwareProject3/PM3_1/data/colleges.json'

# Save the dictionary as a JSON file
with open(output_path, 'w', encoding='utf-8') as file:
    json.dump(data_dict, file, indent=4, ensure_ascii=False)